In [4]:
!pip install -q  langchain-openai==0.1.8 langchain_community==0.2.4 tiktoken==0.7.0 langchain==0.2.3
!pip install -q  google-search-results=2.4.2 langchainhub==0.1.18 wolframalpha==5.1.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.0 MB/s eta 0:00:00


In [5]:
import os
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')
os.environ['SERPAPI_API_KEY'] = userdata.get("serpapi_api_key")
os.environ['WOLFRAM_ALPHA_APPID'] = userdata.get("wolfram_alpha_appid")


In [6]:
from langchain.agents import load_tools
tool_names = ["serpapi","wolfram-alpha"]
tools = load_tools(tool_names)

In [7]:

from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=openai_api_key,model='gpt-4-1106-preview' ,temperature=0)


In [8]:
for tool in tools:
  print("tool 's name: ",tool.name,)
  print("tool 's description", tool.description,'\n')

tool 's name:  Search
tool 's description A search engine. Useful for when you need to answer questions about current events. Input should be a search query. 

tool 's name:  wolfram_alpha
tool 's description A wrapper around Wolfram Alpha. Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Input should be a search query. 



In [9]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
prompt = hub.pull("hwchase17/react")
print(prompt.template)




Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [10]:
dir(tool)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [18]:

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=20)

i = -2
query_list =["""تفاوت لیست و تاپل در پایتون چیست؟""",
     """دمای هوای شهر تهران امروز چند درجه سانتیگراد است؟""",
             """قیمت یک تلویزیون بیست میلیون تومان است اگر 9% ارزش افزوده به آن تعلق بگیرید قیمت آن جقدر میشود؟
             اگر خریدار 30 درصد مبلغ را نقد و بقیه را در 20 قسط مساوی پرداخت کند، هر قسط او چقدر میشود؟""" ]


In [19]:
agent_executor.invoke({"input": query_list[i]})




> Entering new AgentExecutor chain...
The question is asking for the current temperature in Tehran, which is a piece of information that changes frequently and would be best found through a search engine.

Action: Search
Action Input: current temperature in Tehran{'type': 'weather_result', 'temperature': '84', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '13%', 'wind': '7 mph', 'location': 'Tehran, Tehran Province, Iran', 'date': 'Sunday 8:00 AM', 'weather': 'Partly cloudy'}The search result provided the temperature in Fahrenheit. To answer the question, I need to convert this temperature to Celsius.

Action: wolfram_alpha
Action Input: convert 84 Fahrenheit to Celsius

RuntimeError: asyncio.run() cannot be called from a running event loop

In [21]:

!pip install nest_asyncio

# Allow nested asyncio loops
import nest_asyncio
nest_asyncio.apply()

import asyncio  # Make sure to import asyncio

# Move your use of qa_chain into an async function
async def main():
    result = agent_executor.invoke({"input": query_list[i]})
    print(result)

# Run your async function in the existing event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(main())




> Entering new AgentExecutor chain...
The question is asking for the current temperature in Tehran, which is a current event and factual data. I will use the search tool to find the current temperature in Tehran.

Action: Search
Action Input: current temperature in Tehran{'type': 'weather_result', 'temperature': '84', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '13%', 'wind': '7 mph', 'location': 'Tehran, Tehran Province, Iran', 'date': 'Sunday 8:00 AM', 'weather': 'Partly cloudy'}The search result provided the temperature in Fahrenheit. To answer the question, I need to convert the temperature from Fahrenheit to Celsius.

Action: wolfram_alpha
Action Input: convert 84 Fahrenheit to CelsiusAssumption: convert 84 °F (degrees Fahrenheit) to degrees Celsius 
Answer: 28.89 °C (degrees Celsius)I now know the final answer after converting the temperature from Fahrenheit to Celsius.
Final Answer: دمای هوای شهر تهران امروز حدود 29 درجه سانتیگراد است.

> Finished chain.
{'input':

کد بالا فقط برای این نوشته شده که شما بتونید یک

agent_executor

رو در کولب ران کنید اگر در محیط دیگه اای باشه کافیه مستقیم اون رو

invoke

 کنید